# Curves
Curves are abundant in nueroscience. In particular curves you will typically see are exponential decay, sigmoid, logarithmic and linear. Exponential decays occur in PSC/PSPs (both rise and decay), membrane taus (current clamp), and membrane currents (Ih-currents). Sigmoid currents occur in dose-response curves (FI curves and Ih currents). Logarithmic curves occur in membrane vorage changes in spiking activity in cells. Linear curves are seen in IV curves. In neuroscince many of these curves are measuring some sort of dose-response or response-time relationship.

You have seen these curves in action in several chapters such as the [Current Clamp](current_clamp) and [m/sEPSC](mininature_psc). In this chapter we will delve into the specific parameters of the curves and how the curve_fit function in Scipy works.

Some quick basics that all curves typically have. Many curves go from 0 to 1 or 1 to 0. However, most data we collect is not scaled like this. Curves typically have some scaling factor which means you divide or multiple the equation by a number to change the range of possible the equation can output. Curves also have shifting factors which are added or subtracted to the equation and move the equation up or down or to the left or  right. This is really a core concept that I did not pick up early on but wish I had.

In [ ]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import Checkbox, ColumnDataSource, CustomJS, Select, Slider, Spinner
from bokeh.plotting import figure, gridplot
from scipy import optimize

output_notebook()

## Exponential decay
Exponential decay occurs in processes where a value decreases proportionally to its current value. Exponential decay is primarily parameterized by the decay constant, $\lambda$. $\lambda$ can be describe as a time constant, $\tau$ where $\tau = \frac{1}{\lambda}$. $\tau$, the time constant or the time if takes for the process to reach a value of ~1/3 or $\frac{1}{e}$ of the original value. The basic exponential equation is: $N_{0}e^{-\frac{t}{\tau}}$. $N_{0}$ is the original value, and in electrophysiological terms is the amplitude of a PSC/PSP. The equation expects some things. One is that you value decay towards zero. The second is that typically $N_{0}$ is at time point zero. When we curve fit the mEPSCs in the [m/sEPSC](mininature_psc) chapter we created a new time array that started at zero.

The decay equation can be scaled and shifted in several ways. $N_{0}$ can be positive or negative. $N_{0}$ is multiplicative scaling factor, since you leave it out of the equation and you will just get 1 for time value zero since $e^0 = 1$. $\tau$ is a also just a multiplicative scaling factor but for time. You can shift the whole equation up or down by adding a constant if your decay does not converge towards zero but some other constant value. There is no way to just shift the equation along in time. That is because you are assumming $N_{0}$ is $time = 0$ and if you shift $t$ in the equations you change where you are measuring the decay from. To shift the equation in time you just add a constant to your x values after putting them through the equation. You can also add decay equations together to get double, triple or even more decays. However, adding more than 2 or 3 decays together can lead to over fitting and decreased interpretability.

### Single exponential decay

In [ ]:
def exp_decay(x, amplitude=1, tau=1, yshift=0):
    y = yshift + (amplitude * np.exp(-x / tau))
    return y


x = np.arange(300) / 10
source = ColumnDataSource({"x": x, "y": exp_decay(x, amplitude=15, tau=2.5)})

plot = figure(width=400, height=400)

plot.line(x, exp_decay(x, amplitude=15, tau=2.5), line_width=2, line_color="black")
plot.line("x", "y", source=source, line_width=3, line_alpha=0.6, line_color="magenta")

yshift = Slider(start=-10, end=10, value=0, step=0.25, title="Y shift")
xshift = Slider(start=-10, end=10, value=0, step=0.25, title="X shift")
decay_tau = Slider(start=0.75, end=50, value=2.5, step=0.25, title="Decay tau")
amplitude = Slider(start=-30, end=30, value=15, step=0.5, title="Amplitude")
length = Slider(start=10, end=70, value=30, step=0.25, title="Length")

callback = CustomJS(
    args=dict(
        source=source,
        decay_tau=decay_tau,
        amplitude=amplitude,
        length=length,
        yshift=yshift,
        xshift=xshift,
    ),
    code="""
    const len = Math.round(length.value * 10)
    const t_length = Array.from({ length: len }, (_, i) => (xshift.value + i)/10)
    const y = t_length.map(x => {
        return (amplitude.value * Math.exp(-x / decay_tau.value) + yshift.value
    })
    source.data.x = t_length;
    source.data.y = y;
    source.change.emit();
""",
)

yshift.js_on_change("value", callback)
decay_tau.js_on_change("value", callback)
amplitude.js_on_change("value", callback)
length.js_on_change("value", callback)

show(row(plot, column(decay_tau, amplitude, length, yshift, xshift)))

### Double exponential decay
You will see that you can get much more complicated shapes with a double exponential decay. Some shapes don't even look like a regular decay. There are a couple ways you can parameterize the double exponential decay. You could have a y_shift for each decay or just one for both. Usually when we think a curve is a double exponential decay we are assuming that the amplitude of both decays has the same sign. Additionally, you could make the amplitude the same for each decay or allow them to be different. It starts to get very complicated. You can have any combination of factors you choose. For fitting PSC/PSP decays I usually include an amplitude and tau parameter for each decay but ensure that the amplitudes both have the same sign. In the example below I do not ensure the amplitudes are the same sign. I also encourage you to reparameterize the equation if you feel confident enough.

In [ ]:
def db_decay(x, amplitude_fast=1, tau_fast=1, amplitude_slow=0, tau_slow=1):
    y = (amplitude_slow * np.exp(-x / tau_slow)) + (
        amplitude_fast * np.exp(-x / tau_fast)
    )
    return y


x = np.arange(300) / 10
source_db = ColumnDataSource(
    {"x": x, "y": db_decay(x, amplitude_fast=15, tau_fast=2.5)}
)

plot = figure(width=400, height=400)

plot.line(
    x, db_decay(x, amplitude_fast=15, tau_fast=2.5), line_width=2, line_color="black"
)
plot.line(
    "x", "y", source=source_db, line_width=3, line_alpha=0.6, line_color="magenta"
)

tau_slow = Slider(start=0.75, end=50, value=1, step=0.25, title="Tau slow")
amplitude_slow = Slider(start=-30, end=30, value=1, step=0.5, title="Amplitude_slow")
tau_fast = Slider(start=0.75, end=50, value=2.5, step=0.25, title="Tau fast")
amplitude_fast = Slider(start=-30, end=30, value=15, step=0.5, title="Amplitude fast")
length = Slider(start=10, end=70, value=30, step=0.25, title="Length")

callback = CustomJS(
    args=dict(
        source=source_db,
        tau_slow=tau_slow,
        amplitude_slow=amplitude_slow,
        tau_fast=tau_fast,
        amplitude_fast=amplitude_fast,
        length=length,
    ),
    code="""
    const len = Math.round(length.value * 10)
    const t_length = Array.from({ length: len }, (_, i) => (0 + i)/10)
    const y = t_length.map(x => {
        return ((amplitude_slow.value * Math.exp((-x) / tau_slow.value))
        +(amplitude_fast.value * Math.exp((-x) / tau_fast.value)))
    })
    source.data.x = t_length;
    source.data.y = y;
    source.change.emit();
""",
)

tau_slow.js_on_change("value", callback)
amplitude_slow.js_on_change("value", callback)
tau_fast.js_on_change("value", callback)
amplitude_fast.js_on_change("value", callback)
length.js_on_change("value", callback)

show(row(plot, column(tau_slow, amplitude_slow, tau_fast, amplitude_fast, length)))

## Sigmoid curve
You will often see sigmoidal curves in input-output experiments. Some of these experiments include FI curves for firing rate and current density curves for ion channels (like Ih channels). This is because there is often a minimal and maximal amount of voltage/current primarily due the receptors present on a cell as well as the membrane composition. The basic sigmoid curve equation is: $\frac{1}{1+e^{-x}}$. This specific equation is actually the logistic function, but there are other functions that have sigmoidal shapes. You can further parameterize the equation by subtracting/adding a value to x to shift the equation along the x-axis. You can divide x by a number which is the slope. You can multiply the whole equation by a value to scale the output. Lastly you can add a value to the exquation to offset the axis. The full equation looks like this: $\frac{1}{1+e^\frac{x-m}{s}}*y+o$ where $s=slope$, $m=x  offset$, $y=y scale$ and $o=y offset$. You may also see the equation as: $bottom-\frac{bottom-top}{1+10^{(LogEC50-X)*HillSlope}}$.

In [ ]:
def sigmoid(x, max_value, midpoint, slope, offset):
    return 1 / (1 + np.exp((x - midpoint) / slope)) * max_value + offset


x = np.linspace(0, 400)
y1 = sigmoid(x, 16.0, 200.0, -40.0, 0)
y2 = sigmoid(x, 16.0, 200.0, -40.0, 0)

source1 = ColumnDataSource({"x": np.linspace(0, 400), "y1": y1, "y2": y2})
source2 = ColumnDataSource(
    {"x": np.linspace(0, 400, num=49), "y1": np.diff(y1), "y2": np.diff(y2)}
)

plot1 = figure(width=300, height=300)
plot1.line("x", "y1", source=source1, line_width=2, line_alpha=0.6, line_color="black")
plot1.line(
    "x", "y2", source=source1, line_width=3, line_alpha=0.6, line_color="magenta"
)
plot2 = figure(width=300, height=300)
plot2.line("x", "y1", source=source2, line_width=2, line_alpha=0.6, line_color="black")
plot2.line(
    "x", "y2", source=source2, line_width=3, line_alpha=0.6, line_color="magenta"
)

max_val_1 = Slider(value=16.0, start=10.0, end=30.0, step=1.0, title="Response gain 1")
slope_1 = Slider(value=-40, end=-5.0, start=-60.0, step=1.0, title="Input gain 1")
midpoint_1 = Slider(
    value=200.0, start=25.0, end=400.0, step=1.0, title="Current offset 1"
)

callback = CustomJS(
    args=dict(
        source1=source1,
        source2=source2,
        max_val_1=max_val_1,
        slope_1=slope_1,
        midpoint_1=midpoint_1,
    ),
    code="""
    const data = source1.data;
    const x_array = data['x'];
    const y1 = x_array.map((x) => {
        return 1 / (1 + Math.exp((x - midpoint_1.value) / slope_1.value)) * max_val_1.value
    })
    source1.data['y1'] = y1
    const y1_diff = y1.slice(1).map((value, index) => value - y1[index])
    source2.data['y1'] = y1_diff
    source1.change.emit();
    source2.change.emit();
""",
)

max_val_1.js_on_change("value", callback)
slope_1.js_on_change("value", callback)
midpoint_1.js_on_change("value", callback)

show(
    column(
        column(max_val_1, slope_1, midpoint_1),
        gridplot([[plot1, plot2]]),
    )
)

## Logarithmic
The logarithmic curve is pretty much just the exponential decay with some alternative assumptions and a different formula. The logarithmic function seems to closely model how the measures of spike shape, such as the area under the curve, changes as the spike frequency increases. This equation is not often used in neuroscience but I will present it here since it is relevant to the current clamp data we collected. The basic equation is: $C*log(x)$. $C$ is a scaling value and $x$ is the x position. The equation can further modified to shift is in the y direction: $C*log(x)+v$

In [ ]:
def log_func(x, vscale, offset=0):
    return vscale * np.log(x) + offset


x = np.arange(1,301)
source = ColumnDataSource({"x": x, "y": log_func(x, vscale=5)})

plot = figure(width=400, height=400)

plot.line(x, log_func(x, vscale=5), line_width=2, line_color="black")
plot.line("x", "y", source=source, line_width=3, line_alpha=0.6, line_color="magenta")

yshift = Slider(start=-10, end=10, value=0, step=0.25, title="Y shift")
yscale = Slider(start=-10, end=10, value=5, step=0.5, title="Y scale")
length = Slider(start=1, end=600, value=301, step=0.25, title="Length")

callback = CustomJS(
    args=dict(
        source=source,
        yscale=yscale,
        length=length,
        yshift=yshift,
    ),
    code="""
    const len = Math.round(length.value)
    const t_length = Array.from({ length: len }, (_, i) => (1 + i))
    const y = t_length.map(x => {
        return (yscale.value * Math.log(x)+yshift.value)
    })
    source.data.x = t_length;
    source.data.y = y;
    source.change.emit();
""",
)

yshift.js_on_change("value", callback)
yscale.js_on_change("value", callback)
length.js_on_change("value", callback)

show(row(plot, column(yscale, length, yshift)))

## Fitting curves in Python
This next section we will go over how to curve fit in Python. There are two ways to fit curves in Python: optimization-based curve fitting or polynomial curve fitting. If you want values that are interpretable you will want to use the the equations we have provided. If you just want to see if there are different curve shapes you can use the polynomial fit but you will have to run stats on all the coefficients since you will not have a 1 to 1 mapping of what coefficient represents what part of the curve shape. I recommend sticking with the optimization-based methods since it is easy to tweak to get resonable fits.

Curve fitting in Python can be done using the [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) function in the optimization module of Scipy. Curve fit has 3 main parameters you have to pass and two others that are very useful for getting good fits. You will need some function to pass that outputs an array of values. The function must accept an array of x values for the first parameter and have 1 or more parameters that are going to optimized `def test(x, param1, param2, ...)`. You will also need a x array that is passed to the function and y array that the output of the function is compared to. The curve_fit function minimizes the sum of squares alternatively known as least squares. That means it does this `sum((y_output-y_input)**2)` in Python code or in mathematic notation: $\sum_{i=1}^{n}({y_{i}-\hat{y}_{i}})^2$ ($\hat{y}$ is the output of the function).

The two ways that you can improve the curve_fit function output is by providing start values (p0) or by providing and upper and lower bounds (bounds). Some equations you must provide bounds to get a fit since some equations cannot have zeros such as log or equations where you are trying to optimize a divisor (sigmoid function).